# Deep Learning Series


<span style="color:gray">
<br>

1. Neural Net - part.1

</span>

2. <b>Convolution Neural Network</b>


<span style="color:gray">

3. Neural Net - part.2


4. Recursive Nerural Network
</span>

# Convolutional Neural Network

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [2]:
from tqdm import tqdm_notebook

In [3]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(),
                        target_transform=None, download = True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(),
                        target_transform=None, download = True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                          shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                          shuffle=False, num_workers=2, drop_last=True)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 5),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

## Convolution Layer
<img src="img/CNN_01.PNG">

* in_channels ::: 
    - ㅇㅇㅇ


* out_channels ::: 
    - ㅇㅇㅇ


* kernel_size ::: 
    - 커널 사이즈는 필터의 크기로 만일 2로 설정된다면 2x2 사이즈의 필터가 생성된다.


* stride
    - 출력크기를 조정할 목적으로 사용된다.
    - 필터를 적용하는 위치의 간격으로, 스트라이드의 수만큼 필터가 이동하게 된다.


* padding 
    - 출력 크기를 조정할 목적으로 사용된다.
    - 입력 데이터 주변을 특정 값(예컨대 0)으로 채우는 방법이 있다.|

## Pooling Layer
<img src="img/CNN_02.PNG">

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
loss_arr = []
for i in tqdm_notebook(range(num_epoch)):
    for j,[image, label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j%1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3047, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2698, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1802, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0885, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0664, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0909, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0470, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0478, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0478, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0637, device='cuda:0', grad_fn=<NllLossBackward>)



In [7]:
correct = 0
total = 0

with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y_= label.to(device)
        
        output = model.forward(x)
        _, output_index = torch.max(output, 1)
        
        total += label.size(0)
        correct += (output_index == y_).sum().float()
    
    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 98.83814239501953
